<a href="https://colab.research.google.com/github/miamvmian/Scientific-Computation/blob/main/SciComp2024_HW1_ipynb_sol_backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scientific Computing 2024: Homework Assignment 1
Due Sunday October 13, 2024, 23:59

### Problem 1 (2 points)
Under assumptions of Amdahl's law, suppose that 75% of a program are perfectly parallelizable, and the rest is not parallelizable.
1. What is the maximum speedup achievable by parallelization?

$$S=\frac{1}{\frac{3}{4n}+\frac{1}{4}}=\frac{4n}{n+3}$$ where $n$ is the number of cores for parallel computing.

2. Suppose that we have obtained speedup 2 (by using a suitable number of processes). What is the efficiency of this parallelization?
$$S=\frac{4n}{n+3}=2$$the number of computation cores is $n=3$, so the efficiency of the parallelization is:
$$E=\frac{S}{n}=\frac{2}{3}≈67\%$$


### Problem 2 (2 points)
Write a Python or C/C++ program that uses **MPI reduce** to find the largest file in terms of the  number of lines among all .txt files in the working directory (only .txt files should be examined). The program must be callable in the form `mpirun -np <N> python linecount.py` (in the case of the Python version) or `mpirun -np <N> linecount.exe` (the C/C++ version), where `<N>` is the user-defined number of processes. The program is expected to first somehow distribute the files found in the current directory to the processes, then each process should count the lines in the files assigned to it, and finally the result should be MPI-reduced and printed out. The program only needs to output **the number of lines in the largest file**; no need to output the name of this file. The program must work correctly even if the number of files is not divisible by the number of processes.





In [ ]:
!pip install mpi4py

In [ ]:
# create random txt files

import os
import random
import string
from pathlib import Path


def generate_random_string(length: int=10) -> str:
    """Generate a random string of fixed length."""
    letters = string.ascii_letters + string.digits
    return ''.join(random.choice(letters) for _ in range(length))


def create_random_files(num_files: int, min_lines: int, max_lines: int, output_dir: Path)->None:
    """Create a specified number of text files with random lines."""
    # Create output directory if it doesn't exist
    work_path = Path.cwd()
    folder = Path.cwd()/output_dir
    if not folder.exists():
        os.makedirs(work_path/output_dir)

    for i in range(num_files):
        # Generate a random number of lines
        num_lines = random.randint(min_lines, max_lines)

        # Create a filename
        filename = f"file_{i + 1}_{num_lines}.txt"
        filepath = folder/filename

        with open(filepath, 'w') as f:
            for _ in range(num_lines):
                # Write a random string as a line
                line = generate_random_string(random.randint(10, 30))  # Random length between 5 and 20
                f.write(line + '\n')

        print(f"Created {filepath} with {num_lines} lines.")


# Parameters
num_files = random.randint(100, 200)  # Random number of files between 1 and 10
min_lines = 1  # Minimum lines per file
max_lines = 2500  # Maximum lines per file
output_directory = 'random_files'  # Output directory

# Create the files
create_random_files(num_files, min_lines, max_lines, output_directory)

In [ ]:
# linecount.py
from pathlib import Path
from typing import TypeVar, Iterable
from mpi4py import MPI
import sys
import numpy as np

PathLike = TypeVar("PathLike", str, Path)

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
n_cores = comm.Get_size()

# count the number of lines
def count_lines(file_name: Path)->int:
  with open(file_name, "rb") as f:
      num_lines = sum(1 for _ in f)
      return num_lines

if rank == 0:
  folder = "random_files"
  # search files ending with ".txt"
  def search_txt(path: PathLike|str = None) -> Iterable:
      work_path = Path.cwd()
      work_dirs = work_path.parts
      if path is not None:
          dirs = Path(path).parts
          if dirs[0] not in work_dirs:
              data_path = work_path / path
          else:
              indx = work_dirs.index(dirs[0])
              data_path = Path(*work_dirs[0:indx], *dirs)
      else:
          data_path = work_path

      for full_path in data_path.iterdir():
          name = full_path.name
          if name.endswith('.txt'):
              yield full_path

  file_names = list(search_txt(folder))
else:
  file_names = None

file_names = comm.bcast(file_names, root=0)

tot_task = len(file_names)
tasks_perCore = tot_task//n_cores
res_tasks = tot_task%n_cores
file_name = file_names[tasks_perCore*rank:tasks_perCore*(rank+1)]

if res_tasks and rank<=res_tasks-1:
  file_res = file_names[tasks_perCore*n_cores:]
  file_name.append(file_res[rank])

local_max =  np.zeros(1, dtype=int)
glob_max = np.zeros(1, dtype=int)
for fi in file_name:
  num_line = count_lines(fi)
  if num_line>local_max:
    local_max[0] = num_line

comm.Reduce(local_max, glob_max, op=MPI.MAX, root=0)

if rank == 0:
  print(f"The number of lines in the largest file is {glob_max[0]}")


In [77]:
!sudo mpirun --allow-run-as-root --oversubscribe -np 8 python linecount.py

The maximum line number is: 2442




### Problem 3 (2 points)
Solve the Distinct Substrings problem at Sphere online judge: http://www.spoj.com/problems/DISUBSTR/. Provide code passing the test of the judge. Explain how your code works and theoretically estimate the complexity of the algorithm (as $O(f(N))$, where $f(N)$ is some function of the length of the input string).

### Problem 4 (2 points)
Suppose that we want to distribute $N$ personal projects to $N$ students. Assume that each student $(k)_{k=0}^{N-1}$ has a list of his/her preferences for the projects, expressed as a vector $\mathbf r_k$ of integer ranks assigned to each project. Ranks vary between 0 and $N-1$ without repetitions, the **lower** the rank the **more preferable** the project. (For example, the first student's ranks are $\mathbf r_0 = [2,1,0]$, the second's $\mathbf r_1 = [0,2,1]$ and the third $\mathbf r_2 = [0,1,2]$). We want to distribute the projects so as to maximize the total preference, i.e., if $n_k$ denotes the project assigned to the $k$'th student, we want to make $f = \sum_{k=0}^{N-1} \mathbf r_k[n_k]$ as small as possible. (In the example above the optimal distribution is $n_0=2, n_1=0, n_2=1$, which gives $f=1$).  
  * Come up with an algorithm optimizing the distribution and implement it in a Python or C/C++ program. The algorithm should accept the preference vectors and output a recommended distribution $(n_k)_{k=1}^N$. The algorithm need not find the best solution, but is expected to generally produce better solutions than would have been obtained by randomly distributing the projects. The algorithm should be reasonably fast, say run in not more than a few seconds for $N=30$.
  * Compare experimentally your algorithm with the trivial algorithm producing a random distribution. To this end, perform $M=1000$ experiments in each of which 1) random preference vectors for $N=30$ students and projects are generated; 2) the objective function $f$ is evaluated for both algorithms. After finishing all the experiments, plot the two respective distributions of the obtained $M$ values of $f$ and compute the mean values of $f$ for both algorithms.
  
### Problem 5 (2 points)
Suppose that we have developed an algorithm that is supposed to generate independent (quasi-)random numbers uniformly distributed in the interval $[0,1]$. To test our algorithm, we perform a series of experiments. In each experiment, we generate $N=10^3$ numbers $(x_n)_{n=1}^N$ with our algorithm, and compute the minimum distance $r=\min_{1 \le n < m\le N}|x_n-x_m|$ between them. We observe that in more than 99% of such experiments we obtain $r < 10^{-5}$. Does this observation contradict the hypothesis of generating independent uniformly distributed random numbers? Explain your answer.